In [ ]:
!pip install --quiet google-api-python-client google-auth-httplib2 google-auth-oauthlib markdown

In [ ]:
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
from google.auth import default

creds, _ = default(scopes=["https://www.googleapis.com/auth/documents"])
docs_service = build("docs", "v1", credentials=creds)

In [ ]:
MARKDOWN_TEXT = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""


In [ ]:
def create_google_doc(title: str) -> str:
    '''
    Create a Google Doc with the given title.
    
    :param title: The title of the Google Doc to be created.
    :type title: str
    :return: The document ID of the created Google Doc.
    :rtype: str
    '''
    try:
        doc = docs_service.documents().create(
            body={"title": title}
        ).execute()
        return doc["documentId"]
    except Exception as e:
        raise RuntimeError(f"Failed to create document: {e}")

DOC_ID = create_google_doc("Product Team Sync")
print("Document ID:", DOC_ID)


In [ ]:
def insert_text(text, index):
    '''
    Create a request to insert text at a specific index in the document.
    
    :param text: The text to be inserted.
    :param index: The position in the document where the text will be inserted.
    :return: A dictionary representing the insertText request.
    :rtype: dict
    '''
    return {
        "insertText": {
            "location": {"index": index},
            "text": text
        }
    }

def apply_heading(start, end, level):
    '''
    Create a request to apply heading style to a text range.
    
    :param start: The start index of the text range.
    :param end: The end index of the text range.
    :param level: The heading level to apply (e.g., 1 for HEADING_1).
    :return: A dictionary representing the updateParagraphStyle request.
    :rtype: dict
    '''
    return {
        "updateParagraphStyle": {
            "range": {"startIndex": start, "endIndex": end},
            "paragraphStyle": {"namedStyleType": f"HEADING_{level}"},
            "fields": "namedStyleType"
        }
    }

def apply_bold(start, end, color=None):
    '''
    Create a request to apply bold style (and optional color) to a text range.
    
    :param start: The start index of the text range.
    :param end: The end index of the text range.
    :param color: The RGB color to apply to the text (optional).
    :return: A dictionary representing the updateTextStyle request.
    :rtype: dict
    '''
    text_style = {"bold": True}
    fields = ["bold"]

    if color:
        text_style["foregroundColor"] = {
            "color": {
                "rgbColor": color
            }
        }
        fields.append("foregroundColor")
    return {
        "updateTextStyle": {
            "range": {
                "startIndex": start,
                "endIndex": end
            },
            "textStyle": text_style,
            "fields": ",".join(fields)
        }
    }

def apply_footer_style(start, end):
    '''
    Create a request to apply footer style (italic and gray color) to a text range.
    
    :param start: The start index of the text range.
    :param end: The end index of the text range.
    :return: A dictionary representing the updateTextStyle request.
    :rtype: dict
    '''
    return {
        "updateTextStyle": {
            "range": {"startIndex": start, "endIndex": end},
            "textStyle": {
                "italic": True,
                "foregroundColor": {
                    "color": {"rgbColor": {"red": 0.4, "green": 0.4, "blue": 0.4}}
                }
            },
            "fields": "italic,foregroundColor"
        }
    }


In [ ]:
import re

MAGENTA = {"red": 1.0, "green": 0.0, "blue": 1.0}

def render_markdown(doc_id, markdown):
    requests = []
    cursor = 1

    lines = markdown.split("\n")
    for line in lines:
        stripped = line.strip()

        # Title / Headers
        if stripped.startswith("# "):
            text = stripped[2:] + "\n"
            start = cursor
            requests.append(insert_text(text, cursor))
            cursor += len(text)
            requests.append(apply_heading(start, cursor, 1))

        elif stripped.startswith("## "):
            text = stripped[3:] + "\n"
            start = cursor
            requests.append(insert_text(text, cursor))
            cursor += len(text)
            requests.append(apply_heading(start, cursor, 2))

        elif stripped.startswith("### "):
            text = stripped[4:] + "\n"
            start = cursor
            requests.append(insert_text(text, cursor))
            cursor += len(text)
            requests.append(apply_heading(start, cursor, 3))

        # Checkboxes
        elif stripped.startswith("- [ ]"):
            text = stripped[5:] + "\n"
            requests.append(insert_text(text, cursor))
            requests.append({
                "createParagraphBullets": {
                    "range": {"startIndex": cursor, "endIndex": cursor + len(text)},
                    "bulletPreset": "BULLET_CHECKBOX"
                }
            })
            cursor += len(text)

            # Style @mentions
            for match in re.finditer(r"@\w+", text):
                start = cursor - len(text) + match.start()
                end = start + len(match.group())
                requests.append(apply_bold(start, end, color=MAGENTA))

        # Bullets
        elif stripped.startswith(("* ", "- ")):
            text = stripped[2:] + "\n"
            requests.append(insert_text(text, cursor))
            requests.append({
                "createParagraphBullets": {
                    "range": {"startIndex": cursor, "endIndex": cursor + len(text)},
                    "bulletPreset": "BULLET_DISC_CIRCLE_SQUARE"
                }
            })
            cursor += len(text)

        # Footer
        elif stripped.startswith("Meeting recorded"):
            text = stripped + "\n"
            start = cursor
            requests.append(insert_text(text, cursor))
            cursor += len(text)
            requests.append(apply_footer_style(start, cursor))

        elif stripped.startswith("Duration"):
            text = stripped + "\n"
            start = cursor
            requests.append(insert_text(text, cursor))
            cursor += len(text)
            requests.append(apply_footer_style(start, cursor))

        else:
            text = stripped + "\n"
            requests.append(insert_text(text, cursor))
            cursor += len(text)

    docs_service.documents().batchUpdate(
        documentId=doc_id,
        body={"requests": requests}
    ).execute()

render_markdown(DOC_ID, MARKDOWN_TEXT)
print("Markdown successfully converted to Google Doc")